# Homework 7: 1D Spectral Element solver
Author: Will Eaton \
Last updated: Saturday 26th March

## What's in the directory? 
There are quite a few files in this directory so here is a quick guide for any of the things not provided for the homework: 
* ```example_data (directory)``` - this holds some different data from calculations with both homogeneous and heterogeneous element-spacing and material properties. This data is used in plot_example_data.py to produce the animation shown in this report below. 
* ```obj (directory)``` - needs to be there, albeit empty, for compilation 
* ```kricher_codes.py```- originally I wrote a solver in python (called python_alt.py) as a backup to see where I was going wrong. I used some of Lion Krichner's codes for GLL stuff from their Github and these are in this file. 
* ```plot_example_data.py``` - This produces an animation plotting all four different permutations asked for in the homework, on the same figure. It relies on the data in ```example_data```
* ```plot_funcs.py``` - holds a class that does all the leg-work for loading and plotting the data
* ```thermal_profiles.mp4``` - video of the previously-mentioned animation
* ```run.sh``` and ```plot.sh``` - short bash scripts that do make clean, compilation, execution and, in the case of ```plot.sh``` also ```runs plot.py``` to plot the result of the calculation

I've put some notes on running the scripts at the bottom

## Derivations/Equations 

Here I write the derivation for solving the homogenous heat equation, 
$$ \rho c_p\partial_tT - \partial_x (k\partial_x T) = 0 $$
using a spectral element solver. 

Starting with the weak form of the equation: 

$$ \int_0^L \rho c_p w \partial_tT dx = -\int_0^L k \partial_xw \partial_x T dx + [kw \partial_x T]^L_0 $$

where $w$ is a test function. 

Spectral elements in the spatial domain are mapped to a reference element using the mapping 

$$ x(\xi) = \sum_{d=1}^2 N_d(\xi) x_d $$
where $N_d(\xi)$ are linear shape functions in the reference element: 

$$ N_1(\xi) = \frac{1}{2}(1 - \xi)  \: \: \: \: N_2(\xi) = \frac{1}{2}(1 + \xi).$$ 

Hence the geometric mapping has a jacobian that depends on the element width ($h_e$): 

$$ J = \frac{\partial x }{\partial \xi} = \frac{h_e}{2}.$$

Unlike for the geometric mapping, the variable fields (e.g. Temperature) are expressed in a basis of Lagrange polynomials of degree $n$ ($l^n$): 

$$ T(x(\xi), t) = \sum_{b = 0}^N T_{b}(t)\:l_b (\xi),$$
$$ w(x(\xi)) = \sum_{a = 0}^N w_{a}\:l_a (\xi).$$


Note here that the discrete sampling points (finite sum over which the field is expressed, $N$) are GLL points. We can express the weak form equation over the whole domain $x \in [0,L]$ as a summation over each of $n_e$ elements, $e$ that can be individually mapped to a reference element. For a single element the weak form becomes: 

$$\int_{-1}^{+1} \rho c_p w \partial_tT \frac{dx}{d\xi} d\xi = -  \int_{-1}^{+1} k \partial_{\xi}w \partial_{\xi} T \frac{d\xi}{dx} d\xi  + B $$

where B represents the boundary conditions. Subbing in the finite summation approximations for $T$ and $w$ we get: 

$$\sum_{a = 0}^N \sum_{b = 0}^N \int_{-1}^{+1}  \rho c_p w_{a}\:l_a l_b J_e \dot{T}_{b}  d\xi = - \sum_{a = 0}^N \sum_{b = 0}^N  \int_{-1}^{+1} k   w_{a} (\partial_{\xi}l_a)  (\partial_{\xi}l_b) T_{b}  J^{-1}_e d\xi  + B $$

These integrals can be calculated using Gauss quadrature on GLL points: 

$$ \int_{\Omega_e} f(x) dx = \int_{-1}^{+1} f(\xi) J_e d\xi \approx \sum_{\gamma=0}^N W(\xi_{\gamma}) f(\xi_{\gamma}) J(\xi_{\gamma})$$

where $\xi_\gamma$ are the locations discrete GLL points within an element in the referential element domain. Applying these to the above equation for an element we get: 

$$\sum_{a = 0}^N w_{a} \sum_{b = 0}^N \dot{T}_{b} \sum_{\gamma=0}^N W_{\gamma} \rho_{\gamma} {c_p}_{\gamma}  l_a(\xi_{\gamma}) l_b(\xi_{\gamma}) J^{\gamma}_e    = - \sum_{a = 0}^N w_{a}  \sum_{b = 0}^N  T_{b}  \sum_{\gamma=0}^N W_{\gamma} k_{\gamma}   [\partial_{\xi}l_a(\xi_{\gamma}) ]  [\partial_{\xi}l_b(\xi_{\gamma})]  J^{-1}_e  + B $$


Defining a local capacity ($M_{ab}$) and stiffness ($K_{ab}$) as 

$$M_{ab} = \sum_{\gamma=0}^N W_{\gamma} \rho_{\gamma} {c_p}_{\gamma}  l_a(\xi_{\gamma}) l_b(\xi_{\gamma}) J^{\gamma}_e$$
$$ K_{ab} = \sum_{\gamma=0}^N W_{\gamma} k_{\gamma}   [\partial_{\xi}l_a(\xi_{\gamma}) ]  [\partial_{\xi}l_b(\xi_{\gamma})]  J^{-1}_e$$

the summation becomes: 

$$\sum_{a = 0}^N \sum_{b = 0}^N w_{a}  M_{ab}\dot{T}_{b}   = - \sum_{a = 0}^N  \sum_{b = 0}^N  w_{a} K_{ab}T_{b}    + B $$


Importantly, because the discrete sample points ($\xi_{\gamma}$) within each element are GLL points, the Lagrange polynomials have the following property: 
$$ l_a(\xi_i) = \begin{cases}
    1,& i=a\\
    0,              & \text{otherwise}
\end{cases} = \delta_{ia} .$$

Therefore 

$$M_{ab} = \sum_{\gamma=0}^N \rho_{\gamma} {c_p}_{\gamma}  \delta_{ab} J^{\gamma}_e$$
such that $\mathbf{M}$ is a diagonal matrix. It is therefore possible in the time-marching algorithm to treat the capacity matrix as a vector. 


## Simulations 
Below shows the resulting animation from solving the heat equation in four cases: 
* Homogeneous thermal conductivity (k) and homogeneous element spacing (sp)
* Heterogeneous thermal conductivity and homogeneous element spacing 
* Homogenous thermal conductivity and heterogeneous element spacing 
* Heterogeneous thermal conductivity and element spacing 
All simulations are plotted on a single figure for comparison, with ticks showing the location of GLL points:

In [1]:
from IPython.display import Video
Video("thermal_profiles.mp4")

## Running the scripts: 

I slightly edited the ```diffusion.f90``` such that the output directory and name of the snapshots can be edited by the user before running the solver. Using ```run.sh``` will then do all the compilation and execution of the resulting executable. As well as the 'snapshots' two extra files are also outputted in the same output directory: 
* time - this just holds a record of the time for each snapshot - this just makes it easier to load in the data from the snapshots in Python for plotting 
* meta - there are three entries in this file which are (1) the prefix name given by the user for each snapshot (e.g if you dont want to call it snapshot0001 but instead hetero0001) - again this is mostly just so the load process is a bit easier in python (2) the number of snapshots in total (3) the timestep for the simulation 

Because of these two files it means that to load in the data for plotting in python the only thing you need to do is create an object of a class called ```Plotter``` in ```plot_funcs.py``` and pass the directory (e.g. 'example_data/hetero_k_hetero_sp') to the constructor. 